In [1]:
import apache_beam as beam

### ParDo Transform


Takes each element of the PCollection, performs a processing function on it and emits 0,1 o multiple elemens

Is the generic form of the Map and FlatMap phases or Parent Class of them

ParDo is usefull for:

- Filtering 
- Formatting or Type Conversion
- Extracting individual parts
- Computations -> Perform any processing function


When you apply a ParDo Transform you'll need to provide user code in form of a DoFn object

##### ¿What is a DoFn Object?

Beam class that defines a distributed processing function.

It contains all the logic to run  the user provided function parallelly on diferents machines

### ParDo as Map:

In [3]:
class SplitRow(beam.DoFn):
    #Override process function (@process)
    def process(self, element):
        return [element.split(',')]

In [13]:
with beam.Pipeline() as p:
    attendance_count=(
        p
        |   beam.io.ReadFromText('data/dept_data.txt')
        |   beam.ParDo(SplitRow())
        |   beam.Map(lambda record: (record[1], 1))
        |   beam.CombinePerKey(sum)
        |   beam.io.WriteToText('data/out_pardo_map')
    )


In [14]:
!head -n 10 data/out_pardo-00000-of-00001

('Marco', 31)
('Rebekah', 31)
('Itoe', 31)
('Edouard', 31)
('Kyle', 62)
('Kumiko', 62)
('Gaston', 31)
('Beryl', 62)
('Olga', 31)
('Leslie', 31)


### ParDo as FlatMap

###### Bonus: Added filter usin ParDo

In [22]:
class SplitRow(beam.DoFn):
    #Override process function (@process)
    def process(self, element):
        return element.split(',')

In [23]:
class FilterAccountsEmployee(beam.DoFn):
    
    def process(self, element):
        if element[3] == 'Accounts':
            return  [element]

In [24]:
with beam.Pipeline() as p:
    attendance_count=(
        p
        |   beam.io.ReadFromText('data/dept_data.txt')
        |   beam.ParDo(SplitRow())
        |   beam.ParDo(FilterAccountsEmployee())
        |   beam.io.WriteToText('data/out_pardo_flatmap_filter')
    )


IndexError: string index out of range [while running '[24]: ParDo(FilterAccountsEmployee)']

In [18]:
!head -n 10 data/out_pardo_flatmap-00000-of-00001

('4', 275)
('a', 155)
('0', 1078)
('c', 278)
('-', 260)
('1', 179)
('e', 186)
('3', 151)
('t', 31)
('d', 31)
